In [ ]:
# =========================================================
# ✅ CNN-LSTM + Optuna + SHAP 히트맵 + 이상치 제거 + 앙상블
# =========================================================
import os, sys, gc, shap, torch, optuna, platform, logging
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from scipy.stats import zscore
import seaborn as sns

# -------------------------
# 로깅
# -------------------------
sys.path.append(r"C:\ESG_Project1\util")
from logger import setup_logger
logger = setup_logger(__name__)

# -------------------------
# 환경 설정
# -------------------------
TRAIN_CSV = r"C:\ESG_Project1\file\merge_data\train_data.csv"
TEST_CSV  = r"C:\ESG_Project1\file\merge_data\test_data.csv"
COL_Y     = "합산발전량(MWh)"
COL_TIME  = "일시"
COL_PLANT = "발전구분"
NUM_FEATS = ["기온(°C)", "강수량(mm)", "일조(hr)", "일사(MJ/m2)"]
SEQ_LEN, HORIZON = 168, 24
OUTLIER_FRAC = 0.01
SAVE_DIR = r"C:\ESG_Project1\cnn_lstm\output"
os.makedirs(SAVE_DIR, exist_ok=True)
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# =========================================================
# 🧩 데이터 전처리 + 이상치 제거
# =========================================================
def read_csv_auto(path):
    try:
        df = pd.read_csv(path, encoding="utf-8-sig")
    except:
        df = pd.read_csv(path, encoding="cp949")
    return df

def preprocess_data(df):
    for col in NUM_FEATS + [COL_Y]:
        df[col] = df[col].replace([np.inf, -np.inf], np.nan)
    df = df.dropna(subset=NUM_FEATS + [COL_Y])
    df = df[np.abs(zscore(df[COL_Y])) < 3]  # 이상치 제거
    return df.reset_index(drop=True)

train_df = preprocess_data(read_csv_auto(TRAIN_CSV))
test_df  = preprocess_data(read_csv_auto(TEST_CSV))
logger.info(f"Train: {train_df.shape}, Test: {test_df.shape}")

# =========================================================
# 🔧 Feature Scaling
# =========================================================
scaler = MinMaxScaler()
Xtr = scaler.fit_transform(train_df[NUM_FEATS])
Xte = scaler.transform(test_df[NUM_FEATS])
ytr = np.log1p(train_df[COL_Y].values)
yte = np.log1p(test_df[COL_Y].values)
MAX_LOG_Y = float(np.log1p(train_df[COL_Y].quantile(0.999)))

# =========================================================
# 🧠 CNN-LSTM 모델 정의
# =========================================================
class CNNLSTM(nn.Module):
    def __init__(self, n_features, hidden_size=128, num_layers=2, dropout=0.2):
        super().__init__()
        self.conv = nn.Conv1d(n_features, hidden_size, 3, padding=1)
        self.bn = nn.BatchNorm1d(hidden_size)
        self.relu = nn.ReLU()
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc1 = nn.Linear(hidden_size, hidden_size)
        self.drop = nn.Dropout(dropout)
        self.fc2 = nn.Linear(hidden_size, HORIZON)

    def forward(self, x):
        x = x.transpose(1, 2)
        x = self.relu(self.bn(self.conv(x)))
        x = x.transpose(1, 2)
        out, _ = self.lstm(x)
        h = self.drop(self.relu(self.fc1(out[:, -1, :])))
        y = torch.nn.functional.softplus(self.fc2(h))
        return torch.clamp(y, max=MAX_LOG_Y)

# =========================================================
# 📦 Dataset
# =========================================================
class WindowedDataset(Dataset):
    def __init__(self, X, y, seq_len=SEQ_LEN):
        self.X, self.y, self.seq_len = X, y, seq_len
    def __len__(self):
        return len(self.X) - self.seq_len - HORIZON + 1
    def __getitem__(self, idx):
        return (
            torch.tensor(self.X[idx:idx+self.seq_len]).float(),
            torch.tensor(self.y[idx+self.seq_len:idx+self.seq_len+HORIZON]).float()
        )

# =========================================================
# ⚙️ Optuna 탐색
# =========================================================
def objective(trial):
    hidden_size = trial.suggest_int("hidden_size", 64, 256)
    num_layers = trial.suggest_int("num_layers", 1, 3)
    dropout = trial.suggest_float("dropout", 0.1, 0.4)
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)

    model = CNNLSTM(Xtr.shape[1], hidden_size, num_layers, dropout).to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.SmoothL1Loss()

    ds = WindowedDataset(Xtr, ytr)
    loader = DataLoader(ds, batch_size=128, shuffle=True)

    for epoch in range(10):
        model.train()
        for xb, yb in loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            optimizer.zero_grad()
            loss = loss_fn(model(xb), yb)
            loss.backward()
            optimizer.step()

    model.eval()
    with torch.no_grad():
        X_tensor = torch.tensor(Xte[:500]).unsqueeze(1).float().to(DEVICE)
        pred = model(X_tensor).cpu().numpy().ravel()
        y_true = yte[:len(pred)]
        return mean_squared_error(np.expm1(y_true), np.expm1(pred))

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)
best_params = study.best_params
logger.info(f"Best Params: {best_params}")

# =========================================================
# 🧩 모델 학습 (앙상블)
# =========================================================
models = []
for i in range(3):
    model = CNNLSTM(Xtr.shape[1], **best_params).to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=best_params["lr"])
    loss_fn = nn.SmoothL1Loss()
    ds = WindowedDataset(Xtr, ytr)
    loader = DataLoader(ds, batch_size=128, shuffle=True)

    for ep in range(50):
        model.train()
        for xb, yb in loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            optimizer.zero_grad()
            loss = loss_fn(model(xb), yb)
            loss.backward()
            optimizer.step()
    models.append(model)

# =========================================================
# 🔍 예측 + 성능
# =========================================================
def predict(model, X):
    model.eval()
    preds = []
    with torch.no_grad():
        for i in range(0, len(X)-SEQ_LEN-HORIZON, HORIZON):
            xb = torch.tensor(X[i:i+SEQ_LEN]).unsqueeze(0).float().to(DEVICE)
            preds.append(model(xb).cpu().numpy().ravel())
    return np.concatenate(preds)

preds_list = [predict(m, Xte) for m in models]
preds_mean = np.mean(preds_list, axis=0)
y_true = np.expm1(yte[:len(preds_mean)])
pred_inv = np.expm1(preds_mean)

mae = mean_absolute_error(y_true, pred_inv)
rmse = np.sqrt(mean_squared_error(y_true, pred_inv))
r2 = r2_score(y_true, pred_inv)
logger.info(f"Final Ensemble → MAE={mae:.3f} | RMSE={rmse:.3f} | R²={r2:.4f}")

# =========================================================
# 🧠 SHAP 히트맵 (시간 × 특성)
# =========================================================
background = torch.tensor(Xtr[:200]).unsqueeze(1).float().to(DEVICE)
test_sample = torch.tensor(Xte[:200]).unsqueeze(1).float().to(DEVICE)
explainer = shap.DeepExplainer(models[0], background)
shap_values = explainer.shap_values(test_sample)[0]

shap_df = pd.DataFrame(np.mean(np.abs(shap_values), axis=1), columns=NUM_FEATS)
plt.figure(figsize=(10,6))
sns.heatmap(shap_df.T, cmap="RdYlBu_r", annot=False)
plt.title("SHAP Feature Importance (Time × Feature)")
plt.tight_layout()
plt.savefig(os.path.join(SAVE_DIR, "shap_heatmap.png"), dpi=200)
plt.close()

logger.info(f"✅ SHAP 히트맵 저장 완료: {os.path.join(SAVE_DIR, 'shap_heatmap.png')}")


: 